In [16]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
loader = PyPDFDirectoryLoader("./data")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
len(docs)

52

In [19]:
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device":"cpu"}, encode_kwargs={"normalize_embeddings": True})

d:\Machine learning\Generative AI\Langchain\langchain\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
import numpy as np
print(np.array(embeddings.embed_query(docs[0].page_content)))
print(np.array(embeddings.embed_query(docs[0].page_content)).shape)


[ 0.01504208  0.01111973 -0.0143903  ...  0.02992566 -0.01091165
 -0.03430151]
(1024,)


In [21]:
db = FAISS.from_documents(docs, embeddings)

In [22]:
query = "What is positional encoding?"
relevant_documents = db.similarity_search(query)

In [23]:
print(relevant_documents[0].page_content)

tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed. There are many choices of positional encodings,
learned and fixed [9].
In this work, we use sine and cosine functions of different frequencies:
PE(pos,2i)=sin(pos/100002i/d model)
PE(pos,2i+1)=cos(pos/100002i/d model)
where posis the position and iis the dimension. That is, each dimension of the positional encoding
corresponds to a sinusoid. The wavelengths form a geometric progression from 2πto10000 ·2π. We
chose this function because we hypothesized it would allow the model to easily learn to attend by
relative positions, since for any fixed offset k,PEpos+kcan be represented as a linear function of
PEpos.
We also experimented with using learned positional embeddings [ 9] instead, and found that the two


In [24]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B2DD36F160>, search_kwargs={'k': 5})

In [25]:
import os
os.environ["HUGGINGFACE_API_TOKEN"] = os.getenv("HUGGINGFACE_API_TOKEN")

In [26]:
from langchain_community.llms import HuggingFaceEndpoint 

hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2", 
    temperature=0.4,
    max_new_tokens=2048, 
    verbose=True, 
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
    add_to_git_credential=True)
query = "What is positional encoding?"
hf.invoke(query)

WARNING! add_to_git_credential is not default parameter.
                    add_to_git_credential was transferred to model_kwargs.
                    Please make sure that add_to_git_credential is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\FARZIN\.cache\huggingface\token
Login successful


"\n\nPositional encoding is a way to provide the model with information about the position of the input data in the sequence. It is added to the input embeddings to help the model understand the order of the data.\n\nWhy is positional encoding necessary?\n\nTransformer models, which use self-attention mechanisms, do not have an inherent sense of position or order of the input data. They process each input token independently and calculate the attention scores between all pairs of tokens. Positional encoding is added to the input embeddings to help the model understand the relative or absolute position of each token in the sequence.\n\nWhat are the different types of positional encoding?\n\nThere are two main types of positional encoding:\n\n1. Fixed positional encoding: This type of positional encoding uses a set of predefined sinusoidal functions to encode the position of each token. The functions are applied to each dimension of the embedding vector. The functions are learned during 

In [12]:
# Hugging face models run locally thorugh hugging face pipelines
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf_pipeline = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    pipeline_kwargs={"temperature": 0.4, "max_new_tokens": 2048},
)
hf_pipeline.invoke(query)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

d:\Machine learning\Generative AI\Langchain\langchain\lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 

In [59]:
prompt_template = """
Use the following context to generate a response to the following question:
Please try to answer to the question concisely and accurately.
Context: {context}
Question: {question}
"""

In [60]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context,", "question"])

In [61]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [62]:
query = "Differences in the uninsured rate by state in 2022" 

In [64]:
result = retrieval_qa.invoke({"query": query})
print(result['result'])

Answer: The table does not provide information on the uninsured rate by state in 2022. It only shows the results of a language model on English-to-German translation, with metrics such as perplexity and BLEU score.
